**Importing dependencies**

In [18]:
import os
import pandas as pd
import numpy as np
import time
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile

**Tor-Selenium Setup**

In [19]:
torexe = os.popen(r'C:\Users\Melisse\OneDrive - Nanyang Technological University\Desktop\Tor Browser\Browser\TorBrowser\Tor\tor.exe')

firefoxBinary = r"C:\Users\Melisse\OneDrive - Nanyang Technological University\Desktop\Tor Browser\Browser\firefox.exe"
geckodriverPath = r"C:\Users\Melisse\OneDrive - Nanyang Technological University\Desktop\Tor Browser\geckodriver.exe"
proxyIP = "127.0.0.1"
proxyPort = 9150

binary = FirefoxBinary(firefoxBinary)

proxy_settings = {
    "network.proxy.type": 1,
    "network.proxy.socks": proxyIP,
    "network.proxy.socks_port": proxyPort,
    "network.proxy.socks_remote_dns": False,
    "extensions.torlauncher.start_tor": True
}

profile = FirefoxProfile(r'C:\Users\Melisse\OneDrive - Nanyang Technological University\Desktop\Tor Browser\Browser\TorBrowser\Data\Browser\profile.default')
profile.set_preference('extensions.torlauncher.start_tor', True) # bypass tor connection page
profile.set_preference('network.proxy.type', 1)
profile.set_preference('network.proxy.socks', '127.0.0.1')
profile.set_preference('network.proxy.socks_port', 9150)
profile.set_preference("network.proxy.socks_remote_dns", True)
profile.update_preferences()

driver = webdriver.Firefox(executable_path=geckodriverPath, 
                           firefox_binary=binary, 
                           firefox_profile=profile)

C:\Users\Melisse\AppData\Local\Temp\ipykernel_12964\4164202741.py:18: DeprecationWarning: firefox_profile has been deprecated, please use an Options object
  profile = FirefoxProfile(r'C:\Users\Melisse\OneDrive - Nanyang Technological University\Desktop\Tor Browser\Browser\TorBrowser\Data\Browser\profile.default')
C:\Users\Melisse\AppData\Local\Temp\ipykernel_12964\4164202741.py:26: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=geckodriverPath,
C:\Users\Melisse\AppData\Local\Temp\ipykernel_12964\4164202741.py:26: DeprecationWarning: firefox_binary has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=geckodriverPath,
C:\Users\Melisse\AppData\Local\Temp\ipykernel_12964\4164202741.py:26: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  driver = webdriver.Firefox(executable_path=geckodriverPath,


**Scraping Lockbit**

In [20]:
# Wait till your tor browser is connected to the tor network before executing this
driver.get('http://lockbitapt6vx57t3eeqjofwgcglmutr3a35nygvokja5uuccip4ykyd.onion/')

In [21]:
SCROLL_PAUSE_TIME = 5 # as the Tor browser is slow, might need to increase this accordingly

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [24]:
# Define lists to store data for both 'good' and 'bad' categories
companyNames = []
companyDescription = []
dateVictimised = []
noClicks = []
publishedStatus = []

cards_good = driver.find_elements_by_xpath("//div[@class='post-block good']")

# Loop through 'good' cards
for card in cards_good:
    dateVictimisedElement = card.find_element_by_xpath(".//div[@class='views']/div[@class='updated-post-date']//span")
    dateVictimisedText = dateVictimisedElement.text
    dateVictimised.append(dateVictimisedText)

    companyNameElement = card.find_element_by_xpath(".//div[@class='post-title']")
    companyNameText = companyNameElement.text
    companyNames.append(companyNameText)

    companyDescriptionElement = card.find_element_by_xpath(".//div[@class='post-block-body']/div[@class='post-block-text']")
    companyDescriptionText = companyDescriptionElement.text
    companyDescription.append(companyDescriptionText)

    noClicksElement = card.find_element_by_xpath(".//div[@class='views']//span[@style='font-size: 12px; font-weight: bold']")
    noClicksText = noClicksElement.text
    noClicks.append(noClicksText)

    publishedStatus.append('Published')


cards_bad = driver.find_elements_by_xpath("//div[@class='post-block bad']")

# Loop through 'bad' cards
for card in cards_bad:
    dateVictimisedElement = card.find_element_by_xpath(".//div[@class='views']/div[@class='updated-post-date']//span")
    dateVictimisedText = dateVictimisedElement.text
    dateVictimised.append(dateVictimisedText)

    companyNameElement = card.find_element_by_xpath(".//div[@class='post-title']")
    companyNameText = companyNameElement.text
    companyNames.append(companyNameText)

    companyDescriptionElement = card.find_element_by_xpath(".//div[@class='post-block-body']/div[@class='post-block-text']")
    companyDescriptionText = companyDescriptionElement.text
    companyDescription.append(companyDescriptionText)

    noClicksElement = card.find_element_by_xpath(".//div[@class='views']//span[@style='font-size: 12px; font-weight: bold']")
    noClicksText = noClicksElement.text
    noClicks.append(noClicksText)

    publishedStatus.append('Not Published')

driver.quit()




C:\Users\Melisse\AppData\Local\Temp\ipykernel_12964\2419565659.py:8: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  cards_good = driver.find_elements_by_xpath("//div[@class='post-block good']")
C:\Users\Melisse\AppData\Local\Temp\ipykernel_12964\2419565659.py:31: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  cards_bad = driver.find_elements_by_xpath("//div[@class='post-block bad']")


In [28]:

# Create a DataFrame
df = pd.DataFrame({"Company": companyNames, "Company Description": companyDescription, "Date Victimised": dateVictimised, "Number of Clicks": noClicks, "Status": publishedStatus})

# Convert the 'Date Victimised' column to datetime
df['Date Victimised'] = pd.to_datetime(df['Date Victimised'], format='Updated: %d %b, %Y, %H:%M UTC')

# Filter data where the year is less than 2023
date_mask = df['Date Victimised'].dt.year < 2023
df = df[~date_mask]


# Reset the index
df = df.reset_index(drop=True)

# Print the summary of the DataFrame
print("Summary of the DataFrame:")
print(df.info())

# Print the number of rows and columns
num_rows, num_columns = df.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

# Check for missing (empty) cells
missing_cells = df.isnull().sum().sum()
if missing_cells > 0:
    print(f"Number of missing cells: {missing_cells}")
else:
    print("No missing cells in the DataFrame")

# Set the maximum width of the output to a large value to prevent line breaks
pd.set_option('display.width', 1000)
# Ensure all data is displayed
pd.set_option('display.max_rows', None)
print(df)

Summary of the DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998 entries, 0 to 997
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Company              998 non-null    object        
 1   Company Description  998 non-null    object        
 2   Date Victimised      998 non-null    datetime64[ns]
 3   Number of Clicks     998 non-null    object        
 4   Status               998 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 39.1+ KB
None
Number of rows: 998
Number of columns: 5
No missing cells in the DataFrame
                                  Company                                Company Description     Date Victimised Number of Clicks         Status
0                               sirva.com  For media: contact us please. We want to provi... 2023-10-14 10:29:00             6307      Published
1                                cdwg.com

In [29]:
df.to_csv("Lockbit_Data.csv")